<a href="https://colab.research.google.com/github/lukegwoolley/CodeAcademy/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-08-20 15:16:50--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   122MB/s    in 0.2s    

2021-08-20 15:16:51 (122 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:
# add your code here - consider creating a new cell for each section of code
#remove users and books with a low amount of ratings as not statistically significant 
#counts1 = df_ratings['user'].value_counts()
#df_ratings = df_ratings[df_ratings['user'].isin(counts1[counts1 >= 200].index)]
#counts = df_ratings['rating'].value_counts()
#df_ratings = df_ratings[df_ratings['rating'].isin(counts[counts >= 100].index)]

#

combine_book_rating = pd.merge(df_ratings, df_books, on='isbn')
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['title'])
combine_book_rating.isnull().values.any()

book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalrating'})
     [['title', 'totalrating']]
    )
book_ratingCount.head()

,title,totalrating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [10]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalrating >= @popularity_threshold')
rating_popular_book.head()

,user,isbn,rating,title,author,totalrating
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,60
1,2313,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose,60
2,6543,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,60
3,8680,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose,60
4,10314,034545104X,9.0,Flesh Tones: A Novel,M. J. Rose,60


In [11]:
book = "Where the Heart Is (Oprah's Book Club (Paperback))"
print(rating_popular_book[rating_popular_book['title']==book])

         user        isbn  ...        author totalrating
93476  277901  0446672211  ...  Billie Letts         585
93477  277938  0446672211  ...  Billie Letts         585
93478  278144  0446672211  ...  Billie Letts         585
93479  278418  0446672211  ...  Billie Letts         585
93480  278633  0446672211  ...  Billie Letts         585
...       ...         ...  ...           ...         ...
94056  275050  0446672211  ...  Billie Letts         585
94057  275585  0446672211  ...  Billie Letts         585
94058  275809  0446672211  ...  Billie Letts         585
94059  275922  0446672211  ...  Billie Letts         585
94060  276641  0446672211  ...  Billie Letts         585

[585 rows x 6 columns]


In [12]:
rating_popular_book = rating_popular_book.drop_duplicates(['user', 'title'])
rating_popular_book_pivot = rating_popular_book.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
rating_popular_book_matrix = csr_matrix(rating_popular_book_pivot.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(rating_popular_book_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [13]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = model_knn.kneighbors(rating_popular_book_pivot.iloc[rating_popular_book_pivot.index.str.fullmatch(book),:], n_neighbors = 6)

  for i in range(0, len(distances.flatten())):
      if i == 0:
         print('Recommendations for {0}:\n'.format(rating_popular_book_pivot.index[book]))
      else:
         print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_book_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

  return recommended_books

In [17]:
rating_popular_book_matrix.iloc[0,:].reshape(1, -1)

AttributeError: ignored

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [15]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

IndexError: ignored